In [ ]:
#Importing needed libraries
import requests
import pandas as pd
from datetime import datetime
import geopandas as gpd
import fiona
import time
from json import JSONDecodeError
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.discrete.count_model import ZeroInflatedPoisson
from personal_lib import general_functions as gf
import re
from typing import Optional, List, Tuple
import googlemaps
import os
import math
import pyogrio
import geopandas as gpd
import io



creds = gf.get_creds()
google_api_key = creds["Google"]["geocoding"]
open_ai = creds["openai"]["FirstTestKey"]
gmaps = googlemaps.Client(key=google_api_key)

STATE_ABBR = "NY"
pd.options.display.max_columns = 500 
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone_test")

### NYC ENERGY DATA BUILDINGS (### Pulling in the Data Via the APi online Local Law 84) 

In [ ]:
## There are mulitple sources of yearly data. Compiling the source URLs here as well as the API urls to grab
## privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2
building_energy_LL84_sources={"2022+":{
    "api":"https://data.cityofnewyork.us/resource/5zyy-y8am.json",
    "info":"https://data.cityofnewyork.us/Environment/NYC-Building-Energy-and-Water-Data-Disclosure-for-/5zyy-y8am/about_data"
                              },
                              "2021":{
    "api":"https://data.cityofnewyork.us/resource/7x5e-2fxh.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/7x5e-2fxh/about_data"
                              },
                              "2020":{
    "api":"https://data.cityofnewyork.us/resource/usc3-8zwd.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/usc3-8zwd/about_data"
                                  },
                              "2019":{
    "api":"https://data.cityofnewyork.us/resource/wcm8-aq5w.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/wcm8-aq5w/about_data"
                                  },
                              "2018":{
    "api":"https://data.cityofnewyork.us/resource/4tys-3tzj.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4tys-3tzj/about_data"
                                   },
                              "2017":{
    "api":"https://data.cityofnewyork.us/resource/4t62-jm4m.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4t62-jm4m/about_data"
                                  },
                              "2016":{
    "api":"https://data.cityofnewyork.us/resource/utpj-74fz.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/utpj-74fz/about_data"
                                  },
                              "2015":{
    "api":"https://data.cityofnewyork.us/resource/77q4-nkfh.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/77q4-nkfh/about_data"
                                  },
                              "2014":{
    "api":"https://data.cityofnewyork.us/resource/nbun-wekj.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/nbun-wekj/about_data"
                                  },
                              "2013":{
    "api":"https://data.cityofnewyork.us/resource/yr5p-wjer.json",
    "info":"http://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/yr5p-wjer"
                                  },
                              "2012":{
    "api":"https://data.cityofnewyork.us/resource/r6ub-zhff.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/r6ub-zhff/about_data"
                                  },
                              "2011":{
    "api":"https://data.cityofnewyork.us/resource/k7nh-aufb.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/k7nh-aufb/about_data"
                                  },
                              "2010":{
    "api":"https://data.cityofnewyork.us/resource/kswi-37bp.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/kswi-37bp/about_data"
                                  }
}

In [ ]:
PAGE = 1000           
TIMEOUT = 30
MAX_RETRIES = 5
BACKOFF_BASE = 1.5

session = requests.Session()
headers = {}

def fetch_all_rows_1k(api_url: str, source_years: str, source_info_url: str) -> pd.DataFrame:
    offset = 0
    frames = []

    while True:
        params = {"$limit": PAGE, "$offset": offset}

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                resp = session.get(api_url, params=params, headers=headers, timeout=TIMEOUT)
                if resp.status_code in (429, 502, 503, 504):
                    time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))
                    continue
                resp.raise_for_status()

                if "json" not in resp.headers.get("Content-Type", "").lower():
                    preview = resp.text[:200]
                    raise ValueError(f"Non-JSON response (status {resp.status_code}): {preview}")

                data_chunk = resp.json()
                if not data_chunk:
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

                # Create DataFrame
                df = pd.DataFrame(data_chunk)

                # 🔑 Add your metadata columns here
                df["source_years"] = source_years
                df["source_api_url"] = api_url
                df["source_info_url"] = source_info_url

                frames.append(df)

                # If less than PAGE, stop; otherwise keep paginating
                if len(data_chunk) < PAGE:
                    return pd.concat(frames, ignore_index=True)

                offset += PAGE
                break  # Success, go to next page

            except (requests.RequestException, JSONDecodeError, ValueError) as e:
                if attempt == MAX_RETRIES:
                    print(f"⚠️ Failed fetching {api_url} at offset {offset}: {e}")
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
                time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))


In [ ]:
agg_running_list = []
for k, v in building_energy_LL84_sources.items():
    print(f"Fetching {k} -> {v['api']}")
    df = fetch_all_rows_1k(v["api"], k, v["info"])
    if not df.empty:
        agg_running_list.append(df)
    else:
        print(f"Warning: no rows returned for {k} ({v['api']}).")

nyc_building_energy = pd.concat(agg_running_list, ignore_index=True) if agg_running_list else pd.DataFrame()
print(f"Total rows: {len(nyc_building_energy)}")

In [ ]:
### Columns to Drop 
to_drop =[
## Banking oriented COlumns
'bank_branch_computer_density',
 'bank_branch_gross_floor_area',
 'bank_branch_gross_floor_area_ft',
 'bank_branch_number_of',
 'bank_branch_number_of_workers',
 'bank_branch_percent_that',
 'bank_branch_weekly_operating',
 'bank_branch_worker_density',
## College / Uni / School
'college_university_gross',
 'college_university_gross_floor_area_ft',
 'college_university_number',
    'k_12_school_computer_density',
 'k_12_school_cooking_facilities',
 'k_12_school_gross_floor_area',
 'k_12_school_gross_floor_area_ft',
 'k_12_school_high_school',
 'k_12_school_percent_that',
 'k_12_school_refrigeration',
 'k_12_school_weekend_operation',
 'laboratory_gross_floor_area_ft',
    'library_gross_floor_area',
# OTher
"automobile_dealership_gross",
'convenience_store_without',
'data_center_energy_estimates_applied',
 'data_center_gross_floor_area',
 'data_center_gross_floor_area_ft',
 'data_center_it_energy',
 'data_center_it_energy_configuration',
 'data_center_it_equipment_input_meter_kwh',
 'data_center_it_site_energy',
 'data_center_it_site_energy_kwh',
 'data_center_it_source_energy_kbtu',
 'data_center_national_median',
 'data_center_pdu_input_meter_kwh',
 'data_center_pdu_output_meter_kwh',
 'data_center_ups_output_meter_kwh',
 'enclosed_mall_gross_floor',
 'enclosed_mall_gross_floor_area_ft',
 'fast_food_restaurant_gross',
 'financial_office_gross_floor',
 'financial_office_gross_floor_area_ft',
 'financial_office_number_of',
 'financial_office_number_of_1',
 'financial_office_number_of_computers',
 'financial_office_number_of_workers_on_main_shift',
 'financial_office_weekly',
 'financial_office_weekly_operating_hours',
 'fitness_center_health_club',
 'fitness_center_health_club_gym_gross_floor_area_ft',
 'food_sales_gross_floor_area',
 'food_sales_gross_floor_area_ft',
 'food_service_gross_floor',
 'food_service_gross_floor_area_ft',
     'worship_facility_computer',
 'worship_facility_cooking',
 'worship_facility_gross_floor',
 'worship_facility_gross_floor_area_ft',
 'worship_facility_weekly',
    'supermarket_grocery_cooking',
 'supermarket_grocery_gross',
 'supermarket_grocery_gross_floor_area_ft',
 'supermarket_grocery_number',
 'supermarket_grocery_number_1',
 'supermarket_grocery_number_2',
 'supermarket_grocery_number_of_open_or_closed_refrigeration_freezer_units',
 'supermarket_grocery_number_of_walk_in_refrigeration_freezer_units',
 'supermarket_grocery_percent',
 'supermarket_grocery_walk',
 'supermarket_grocery_weekly',
 'supermarket_grocery_worker',
 'swimming_pool_approximate',
 'swimming_pool_location_of',
 'swimming_pool_months_in_use',
     'social_meeting_hall_gross',
 'social_meeting_hall_gross_floor_area_ft',
     'residence_hall_dormitory',
 'residence_hall_dormitory_1',
 'residence_hall_dormitory_2',
 'residence_hall_dormitory_3',
 'residence_hall_dormitory_4',
 'residence_hall_dormitory_gross_floor_area_ft',
 'restaurant_gross_floor_area',
 'restaurant_gross_floor_area_ft',
 'restaurant_weekly_operating',
 'restaurant_weekly_operating_hours',
 'restaurant_worker_density',
 'restaurant_worker_density_number_per_1_000_sq_ft',
 'retail_store_cash_register',
 'retail_store_computer_density',
 'retail_store_exterior_entrance',
 'retail_store_gross_floor',
 'retail_store_gross_floor_area_ft',
 'retail_store_number_of_open',
 'retail_store_number_of_open_or_closed_refrigeration_freezer_units',
 'retail_store_number_of_walk',
 'retail_store_number_of_walk_in_refrigeration_freezer_units',
 'retail_store_open_or_closed',
 'retail_store_percent_that',
 'retail_store_walk_in',
 'retail_store_weekly_operating',
 'retail_store_worker_density',
 'self_storage_facility_gross',
 'self_storage_facility_gross_floor_area_ft',
 'senior_care_community_average',
 'senior_care_community_gross',
 'senior_care_community_maximum',
 'senior_care_community_number',
 'senior_care_community_number_1',
 'senior_care_community_number_2',
 'senior_care_community_number_3',
 'senior_care_community_number_4',
 'senior_care_community_number_5',
 'senior_care_community_number_6',
 'senior_care_community_percent',
 'senior_living_community_gross_floor_area_ft',
 'senior_living_community_living_unit_density_number_per_1_000_sq_ft',
    'movie_theater_gross_floor_area_ft',
    'non_refrigerated_warehouse',
 'non_refrigerated_warehouse_1',
 'non_refrigerated_warehouse_2',
 'non_refrigerated_warehouse_3',
 'non_refrigerated_warehouse_4',
 'non_refrigerated_warehouse_5',
 'non_refrigerated_warehouse_gross_floor_area_ft',
     'parking_completely_enclosed',
 'parking_completely_enclosed_parking_garage_size_ft',
 'parking_gross_floor_area',
 'parking_gross_floor_area_ft',
 'parking_open_parking_lot',
 'parking_open_parking_lot_size_ft',
 'parking_partially_enclosed',
 'parking_partially_enclosed_parking_garage_size_ft',
     'strip_mall_gross_floor_area',
## Hostpial / Hotel
    'hospital_general_medical',
 'hospital_general_medical_1',
 'hospital_general_medical_10',
 'hospital_general_medical_11',
 'hospital_general_medical_12',
 'hospital_general_medical_13',
 'hospital_general_medical_14',
 'hospital_general_medical_15',
 'hospital_general_medical_16',
 'hospital_general_medical_17',
 'hospital_general_medical_2',
 'hospital_general_medical_3',
 'hospital_general_medical_4',
 'hospital_general_medical_5',
 'hospital_general_medical_6',
 'hospital_general_medical_7',
 'hospital_general_medical_8',
 'hospital_general_medical_9',
 'hotel_amount_of_laundry',
 'hotel_cooking_facilities',
 'hotel_full_service_spa_floor',
 'hotel_gross_floor_area_ft',
 'hotel_gym_fitness_center',
 'hotel_gym_fitness_center_floor_area_ft',
 'hotel_number_of_rooms',
 'hotel_percent_that_can_be',
 'hotel_room_density_number',
 'hotel_type_of_laundry_facility',
 'hotel_worker_density_number',
 'urgent_care_clinic_other',
 'urgent_care_clinic_other_outpatient_gross_floor_area_ft',
    'mailing_center_post_office_gross_floor_area_ft',
 'manufacturing_industrial_plant_gross_floor_area_ft',
 'medical_office_gross_floor',
 'medical_office_gross_floor_area_ft',
 'medical_office_mri_machine',
 'medical_office_number_of',
 'medical_office_number_of_1',
 'medical_office_number_of_computers',
 'medical_office_number_of_mri_machines',
 'medical_office_number_of_workers_on_main_shift',
 'medical_office_percent_that',
 'medical_office_percent_that_1',
 'medical_office_percent_that_can_be_cooled',
 'medical_office_percent_that_can_be_heated',
 'medical_office_weekly',
 'medical_office_weekly_operating_hours',
    'museum_gross_floor_area_ft',
    'office_computer_density_number',
 'office_gross_floor_area_ft',
 'office_number_of_computers',
 'office_number_of_workers',
 'office_number_of_workers_on_main_shift',
 'office_percent_that_can_be',
 'office_percent_that_can_be_1',
 'office_percent_that_can_be_cooled',
 'office_percent_that_can_be_heated',
 'office_weekly_operating_hours',
 'office_worker_density_number',
 'office_worker_density_number_per_1_000_sq_ft',
     'adult_education_gross_floor',
 'adult_education_gross_floor_area_ft',
    'data_center_ups_output_meter',
 'data_center_pdu_input_meter',
 'data_center_pdu_output_meter',
 'data_center_it_equipment',
 'data_center_it_site_energy',
 'data_center_it_source_energy',
 'data_center_pue',
 'data_center_national_median',
 'data_center_gross_floor_area',
 'data_center_ups_system',
 'data_center_it_energy',
 'data_center_cooling_equipment',
'supermarkets_grocery_gross',
 'supermarkets_grocery_number',
 'supermarkets_grocery_number_1',
 'supermarkets_grocery_number_2',
 'supermarkets_grocery_percent',
 'supermarkets_grocery_presence',
 'supermarkets_grocery_walk',
 'supermarkets_grocery_weekly',
 'supermarkets_grocery_workers',
'house_of_worship_gross_floor',
 'house_of_worship_pc_density',
 'house_of_worship_weekly',
 'house_of_worship_presence',
'residence_halls_dormitories',
'medical_office_percent_cooled',
 'residence_halls_dormitories_1',
 'residence_halls_dormitories_2',
 'residence_halls_dormitories_3',
 'residence_halls_dormitories_4',
'hotel_onsite_laundry_short',
 'warehouse_unrefrigerated',
 'warehouse_unrefrigerated_1',
 'warehouse_unrefrigerated_2',
 'warehouse_unrefrigerated_3',
 'warehouse_unrefrigerated_4',
 'warehouse_unrefrigerated_5',
 'warehouse_unrefrigerated_6',
 'hospital_gross_floor_area',
 'hospital_laboratory_y_1_n',
 'hospital_laundry_facility',
 'hospital_maximum_number_of',
 'hospital_number_of_buildings',
 'warehouse_refrigerated_gross',
 'warehouse_refrigerated_weekly',
 'warehouse_refrigerated_workers',
 'hospital_number_of_licensed',
 'multifamily_home_dishwashers',
 'fuel_oil_2_use_kbtu',
 'fuel_oil_4_use_kbtu',
 'diesel_2_use_kbtu',
 'district_steam_use_kbtu',
 'natural_gas_use_kbtu',
'natural_gas_use_therms',
 'difference_from_national',
 'difference_from_national_1',
 'current_direct_ghg_emissions',
 'current_indirect_ghg_emissions',
'office_gross_floor_area_sq',
 'office_office_air_conditioned',
 'office_pc_density',
 'office_workers_density',
 'parking_enclosed_floor_area',
 'parking_non_enclosed_floor',
 'parking_open_floor_area_w',
 'parking_weekly_hours_of_access',
 'retail_gross_floor_area_sq',
 'retail_cash_register_density',
 'retail_exterior_entrance',
 'retail_workers_density',
 'retail_weekly_operating_hours',
 'retail_walk_in_refrig_density',
 'retail_percent_cooled',
 'retail_pc_density',
 'retail_open_closed_refrig',
 'retail_number_of_walk_in',
 'retail_number_of_open_or',
'data_center_annual_it_energy',
'other_space_type_name',
 'other_gross_floor_area_sq',
 'other_number_of_pcs',
 'other_workers_on_main_shift',
 'hotel_gross_floor_area_sq',
 'hotel_room_density',
 'hotel_percent_cooled',
 'hotel_presence_of_cooking',
 'hotel_workers_density',
 'bank_financial_institution',
 'bank_financial_institution_1',
 'bank_financial_institution_2',
 'bank_financial_institution_3',
 'bank_financial_institution_4',
 'bank_financial_institution_5',
 'k_12_school_high_school_y',
 'k_12_school_open_weekends',
 'k_12_school_pc_density',
 'k_12_school_percent_cooled',
 'k_12_school_walk_in_refrig',
 'k_12_school_presence_of',
 'swimming_pool_size',
 'swimming_pool_indoor_outdoor',
 'hotel_floor_area_of_full',
 'hotel_floor_area_of_gym',
 'hotel_average_occupancy',
 'hotel_quantity_of_laundry',
'number_of_staffed_beds',
 'number_of_mri_machines',
 'senior_care_facility_average',
 'senior_care_facility_gross',
 'senior_care_facility_number',
 'senior_care_facility_number_1',
 'senior_care_facility_number_2',
 'senior_care_facility_number_3',
 'senior_care_facility_number_4',
 'senior_care_facility_percent',
 'senior_care_facility_total',
 'senior_care_facility_total_1',
 'senior_care_facility_workers',]

In [ ]:
chunksize = 100000  
filtered_chunks = []
for chunk in pd.read_csv("nyc_dob_energy_2010_2024.csv", chunksize=chunksize, usecols=lambda col: col not in to_drop):
    ## Using nulls instead of "not Available".
        ## Using nulls instead of "not Available".
    chunk = chunk.replace("Not Available",np.nan)
    # Convert year_ending to datetime
    chunk["year_ending"] = pd.to_datetime(chunk["year_ending"], errors="coerce")
    # Optional: extract just the year if that’s all you need
    chunk["year_ending_year"] = chunk["year_ending"].dt.year
    # print(chunk["year_ending_year"].unique())
    
    ### Property Types
    chunk["primary_property_type"] = chunk["primary_property_type"].combine_first(chunk["primary_property_type_epa"])
    chunk["primary_property_type_self"] = chunk["primary_property_type_self"].combine_first(chunk["primary_property_type_self_selected"])
    chunk = chunk.drop(columns=["primary_property_type_self_selected","primary_property_type_epa"])


    ## Beginning of Limiting to Multifamily Homes (Step 1)
    residential_chunk = chunk[(((chunk["list_of_all_property_use"].isin([i for i in chunk["list_of_all_property_use"].unique() if 'Multifamily Housing' in str(i)]))
            |(chunk["list_of_all_property_use"].isnull()))
           &((chunk["primary_property_type_self"]=="Multifamily Housing")|(chunk["primary_property_type_self"].isnull()))
           &((chunk["primary_property_type"]=="Multifamily Housing")|(chunk["primary_property_type"].isnull())))]
    
    ## Second Step is limiting to NON MIXED USE, so ONLY MultiFamily Residences
    residential_chunk2 = residential_chunk[((residential_chunk['list_of_all_property_use'].isnull())
                                            | residential_chunk['list_of_all_property_use'].apply(
                                                lambda x: len(x.split(",")) == 1 if isinstance(x, str) else False
                                            ))]


    ## Limtiing to the Metered Areas for whole property or whole building
    residential_chunk3 = residential_chunk2[((residential_chunk2["metered_areas_energy"].isin(['Whole Building', "Whole Property"]))
                                             |(residential_chunk2["metered_areas_energy"].isnull()))]

    residential_chunk3 = residential_chunk3[((residential_chunk3["construction_status"]=='Existing')|(residential_chunk3["construction_status"].isnull()))]
    
    ## Stand Alone Properties
    residential_chunk3 = residential_chunk3[((residential_chunk3["parent_property_id"].isnull())
                                             |(residential_chunk3["parent_property_id"]=='Not Applicable: Standalone Property'))]
    ## Dropping what i already filtered on
    residential_chunk3 = residential_chunk3.drop(columns=["primary_property_type_self","primary_property_type",
                                                          "national_median_reference", "list_of_all_property_use",
                                                          "largest_property_use_type","construction_status",
                                                          "parent_property_id","parent_property_name"])
    
    # ## Fruther Limiting to those that are under 10 Stories. 
    # residential_chunk2 = residential_chunk2.replace("Not Available",np.nan).dropna(how='all',axis=1)
    # residential_chunk2 = residential_chunk2.dropna(how='all',axis=1)

    filtered_chunks.append(residential_chunk3)

residential3 = pd.concat(filtered_chunks, ignore_index=True)

### Fixing Via Self Joins and Otherwise for Earlier years in data without Property ID / BBL

In [ ]:
## Finalize the truth set year column 
residential3["year_ending_year"] = residential3["year_ending_year"].combine_first(residential3["source_years"]).astype(int)
### Consolidating those columns that contain potential ID numbers
residential3["bbl"] = residential3["nyc_borough_block_and_lot"].combine_first(residential3['nyc_borough_block_and_lot_bbl']).combine_first(residential3["bbl10"]).combine_first(residential3["bbl"])
residential3.loc[residential3["bbl"].notna(), "bbl"] = residential3.loc[residential3["bbl"].notna(), "bbl"].astype(str).str.replace(r"\.0$", "", regex=True).str.replace(r"-", "", regex=False).str.replace(r"/", "", regex=False).str.replace(r".", "", regex=False)
residential3["postcode"] = residential3["postcode"].combine_first(residential3["postal_code"])
residential3['postcode'] = residential3['postcode'].apply(lambda x : str(x).replace(",0",'') if x else x)
residential3 = residential3.drop(columns=["postal_code","nyc_borough_block_and_lot",'nyc_borough_block_and_lot_bbl',"bbl10"])

In [ ]:
### Checking the row numbers and Property Id vals
residential3.groupby(["year_ending_year"]).agg({"property_id":["nunique","count"],"source_api_url":"count"})

In [ ]:
### Attempting to self-enrich data for property ids
null_property_ids = residential3[residential3["property_id"].isnull()]
print(null_property_ids.shape)
property_ids = residential3[~residential3["property_id"].isnull()]
print(property_ids.shape)

In [ ]:
### immediate Crosswalk for BBL and Property IDs Merging 
cw_join = property_ids[["property_id","bbl"]][~property_ids["bbl"].isnull()].drop_duplicates()
null_property_ids_enr1 = null_property_ids[[c for c in null_property_ids.columns if c !="property_id"]].merge(cw_join,how='left',on=["bbl"])
unresolved = null_property_ids_enr1[null_property_ids_enr1["property_id"].isnull()]
resolved = null_property_ids_enr1[~null_property_ids_enr1["property_id"].isnull()]
property_ids = pd.concat([property_ids,resolved]).drop_duplicates()

In [ ]:
## Taking a look
property_ids.groupby(["property_id"]).agg({"bbl":["nunique","count"],"source_api_url":"count"})

In [ ]:
property_ids.groupby(["year_ending_year"]).agg({"property_id":["nunique","count"],"source_api_url":"count"})

In [ ]:
### Resolving Mulitple BBLS per property Id
test = property_ids[["property_id", "bbl", "address_1", "city","latitude", "longitude", "postcode", "year_ending_year"]].drop_duplicates()
test2 = (test.groupby("property_id").agg(bbl_nunique=("bbl", "nunique"),address_nunique=("address_1", "nunique"),
                                         year_min=("year_ending_year", "min"),year_max=("year_ending_year", "max"),).reset_index())
to_fix = test2[test2["bbl_nunique"] > 1]
to_fix = to_fix[((to_fix["year_min"]==2010)
        &(to_fix["year_max"]>=2017))]

In [ ]:
for i, r in to_fix.iterrows():
    print(r["property_id"])
    temp_df = property_ids[["property_id","bbl","year_ending","address_1","city","postcode"]][property_ids["property_id"]==r['property_id']]
    print(temp_df['bbl'].unique())
    #Most frequent (mode) BBL for this property_id
    mode_bbl = temp_df['bbl'].mode(dropna=True).iloc[0]
    if ";" in mode_bbl:
        continue
    else:
        print(temp_df.shape)
        # Write that BBL back onto ALL rows for this property_id
        property_ids.loc[property_ids["property_id"] == r['property_id'], "bbl"] = mode_bbl

In [ ]:
### Putting together Processed Results So Far 
## Keepgin the unresolved with BBL values, dropping those that are null because BBL and Property ID are null.
print(unresolved[unresolved["bbl"].isnull()].shape)
print(unresolved[~unresolved["bbl"].isnull()].shape)
# test = pd.concat()
keep = unresolved[~unresolved["bbl"].isnull()]
keep["resolved_flag"]="UNRESOLVED"
property_ids["resolved_flag"]="RESOLVED"
residential3_refined= pd.concat([property_ids,keep])

#### Limiting by Years and for those Buildigns that have been in the Data for the years we want

In [ ]:
buildings_2010_property_ids = residential3_refined[((residential3_refined["year_ending_year"].isin([2010,2011]))
                                                    &(~residential3_refined["property_id"].isnull()))]["property_id"].unique()
buildings_2010_bbls = residential3_refined[((residential3_refined["year_ending_year"].isin([2010,2011]))
                                            &(~residential3_refined["bbl"].isnull()))]["bbl"].unique()
print("CheckingGrouping - Prop ID")
print(residential3_refined[residential3_refined['property_id'].isin(buildings_2010_property_ids)].groupby(['year_ending_year']).agg({"address_1":"count"}).reset_index())
print("CheckingGrouping - BBL")
print(residential3_refined[residential3_refined['bbl'].isin(buildings_2010_bbls)].groupby(['year_ending_year']).agg({"address_1":"count"}).reset_index())
### Limtiing to 2017 and before 
residential3_refined_b4_2018 = residential3_refined[residential3_refined["year_ending_year"]<2018]
print("YEARS")
print(residential3_refined_b4_2018['year_ending_year'].unique())
residential3_2010_2017 = residential3_refined_b4_2018[((residential3_refined_b4_2018['property_id'].isin(buildings_2010_property_ids))
                                                       |(residential3_refined_b4_2018['bbl'].isin(buildings_2010_bbls)))]



In [ ]:
### Building Info Only 
building_info =residential3_2010_2017[["property_id",'bbl',"address_1","address_2","city","postcode",'county','borough',"latitude","longitude"]].drop_duplicates()

#### Cleaning Address, Prepping for Geocoding where needed.

In [ ]:
# NYC biasing bounds (rough NYC box)
NYC_BOUNDS = {
    "southwest": {"lat": 40.477399, "lng": -74.259090},
    "northeast": {"lat": 40.917577, "lng": -73.700272},
}

# Common misspellings / normalizations seen in your data
COMMON_FIXES = {
    r"\bWASHIGNTON\b": "WASHINGTON",
    r"\bWASHNGTN\b": "WASHINGTON",
    r"\bALBERMALE\b": "ALBEMARLE",
    r"\bALBERMALE\b": "ALBEMARLE",
    r"\bAMSETRDAM\b": "AMSTERDAM",
    r"\bKINGBRIDGE\b": "KINGSBRIDGE",
    r"\bOVINGTON\b": "OVINGTON",   # keep as is (looks fine)
    r"\bNAGLE HOUSE\b": "NAGLE AVE",  # likely street not building name
    r"\bNAGLE\b": "NAGLE",  # pass-through
    r"\bST\s*NICOLAS\b": "ST NICHOLAS",
    r"\bOILVER\b": "OLIVER",
    r"\bRIVER SIDE\b": "RIVERSIDE",
    r"\bSTEET\b": "STREET",
    r"\bSREET\b": "STREET",
    r"\bFOR WASHINGTON\b": "FORT WASHINGTON",
    r"\bFT\.?\b": "FORT",
    r"\bBIVONA\b": "BIVONA",  # leave
    r"\bCLARKE\b": "CLARKE",  # leave
    r"\bCAROLL\b": "CARROLL",
    r"\bCLAFFIN\b": "CLAFLIN",
    r"\bDEKALB\b": "DEKALB",
    r"\bLAFAYETTE\b": "LAFAYETTE",
    r"\bW\s*MOSHOLU\b": "W MOSHOLU",
    r"\bE\s*MOSHOLU\b": "E MOSHOLU",
    r"\bMOSHOLU\b": "MOSHOLU",
    r"\bPARSON\b": "PARSONS",
    r"\bPARSONS BLVD?\b": "PARSONS BLVD",
}

# Street type vocabulary to help "best-guess" if clearly missing
STREET_TYPES = ["ST", "AVE", "RD", "BLVD", "PL", "LN", "DR", "CT", "PKWY", "TER", "PLZ"]

def _norm_ws(s: Optional[str]) -> Optional[str]:
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return None
    s = str(s)
    # HTML escapes commonly seen
    s = s.replace("&amp;", "&")
    s = s.replace("\u2013", "-").replace("\u2014", "-")
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s if s else None

def _apply_common_fixes(u: str) -> str:
    for pat, repl in COMMON_FIXES.items():
        u = re.sub(pat, repl, u, flags=re.IGNORECASE)
    return u

def _std_unit(s: Optional[str]) -> Optional[str]:
    u = _norm_ws(s)
    if not u:
        return None
    u = u.upper().replace("#", "").strip()
    if not u:
        return None
    u = re.sub(r"^(APARTMENT|APT\.?)\s*", "APT ", u)
    u = re.sub(r"^(SUITE|STE\.?)\s*", "STE ", u)
    u = re.sub(r"^(FLOOR|FL\.?)\s*", "FL ", u)
    if not re.match(r"^(APT|STE|FL)\b", u):  # bare "5B" → "APT 5B"
        u = f"APT {u}"
    return u

def _std_street(u: Optional[str]) -> Optional[str]:
    u = _norm_ws(u)
    if not u:
        return None
    u = _apply_common_fixes(u.upper())

    repl = {
        " STREET": " ST",
        " AVENUE": " AVE",
        " ROAD": " RD",
        " BOULEVARD": " BLVD",
        " PLACE": " PL",
        " LANE": " LN",
        " DRIVE": " DR",
        " COURT": " CT",
        " PARKWAY": " PKWY",
        " TERRACE": " TER",
        " PLAZA": " PLZ",
    }
    for k, v in repl.items():
        u = re.sub(k + r"\b", v, u)

    # Normalize ordinals and add missing "TH/ST/ND/RD" on bare numbers like "E 48" or "West104"
    u = re.sub(r"\b(\d+)\s*(ST|ND|RD|TH)\b", lambda m: f"{int(m.group(1))}{m.group(2)}", u)
    u = re.sub(r"\b(WEST|W|EAST|E)\s*(\d{1,3})(?=\b)", r"\1 \2", u)  # ensure space
    def add_ordinal(m):
        n = int(m.group(2))
        suf = "TH"
        if n % 10 == 1 and n % 100 != 11: suf = "ST"
        elif n % 10 == 2 and n % 100 != 12: suf = "ND"
        elif n % 10 == 3 and n % 100 != 13: suf = "RD"
        return f"{m.group(1)} {n}{suf}"
    u = re.sub(r"\b(WEST|W|EAST|E)\s+(\d{1,3})\b(?!\s*(ST|AVE|RD|BLVD|PL|LN|DR|CT|PKWY|TER|PLZ))", add_ordinal, u)

    # Collapse multiple addresses to last one here (we also split earlier in the pipeline)
    if "/" in u:
        u = u.split("/")[-1].strip()

    # Queens: hyphenated house numbers like "41-07 42ND ST" are valid; leave them as-is
    # Best-guess add a street type if line is like "308 WEST 104TH" (no type) or "160 E 48TH"
    if re.search(r"\b(WEST|W|EAST|E|NORTH|N|SOUTH|S)\b", u) and re.search(r"\b\d{1,3}(ST|ND|RD|TH)\b", u) and not re.search(r"\b(ST|AVE|RD|BLVD|PL|LN|DR|CT|PKWY|TER|PLZ)\b", u):
        u = u + " ST"

    # Convert key phrases
    u = re.sub(r"\bFT\b", "FORT", u)

    return u.title()

def _borough_from_fields(borough, county, city) -> Optional[str]:
    vals = " ".join([str(x) for x in [borough, county, city] if pd.notna(x)]).upper()
    if any(x in vals for x in ["MANHATTAN", "NEW YORK", "NY COUNTY"]): return "MANHATTAN"
    if "BRONX" in vals: return "BRONX"
    if any(x in vals for x in ["BROOKLYN", "KINGS"]): return "BROOKLYN"
    if "QUEENS" in vals: return "QUEENS"
    if any(x in vals for x in ["STATEN", "RICHMOND"]): return "STATEN ISLAND"
    return None

def _usps_city_from_borough(city, borough_norm) -> Optional[str]:
    c = (_norm_ws(city) or "").upper()
    if borough_norm == "MANHATTAN": return "New York"
    if borough_norm == "BRONX": return "Bronx"
    if borough_norm == "BROOKLYN": return "Brooklyn"
    if borough_norm == "QUEENS":
        # USPS accepts neighborhoods, but "Queens" is safest default
        return "Queens" if c in ["", "QUEENS", "NAN"] else c.title()
    if borough_norm == "STATEN ISLAND": return "Staten Island"
    return c.title() if c else None

def _std_zip5(z):
    if z is None or (isinstance(z, float) and pd.isna(z)):
        return None
    s = re.sub(r"\D", "", str(z))
    return s[:5] if len(s) >= 5 else None

# ---------- NEW: address candidate generator ----------

SEP_PATTERN = re.compile(r"\s*(?:;|,|/|&{1,2}| and | AND |\bETAL\b|\bet al\b|\(|\)|\bAKA\b|\bA\/K\/A\b)\s*", flags=re.IGNORECASE)

def _is_pobox(s: str) -> bool:
    return bool(re.search(r"\bP\.?\s*O\.?\s*BOX\b", s, flags=re.IGNORECASE))

def _has_street_name(u: str) -> bool:
    # needs something besides just a house number or zeros
    return bool(re.search(r"\b[A-Z][A-Z]+\b", u)) and not re.fullmatch(r"\d+(-\d+)?", re.sub(r"[^0-9\-]", "", u or ""))

def _expand_number_ranges(u: str) -> List[str]:
    """
    '2078-84-90 Morris Ave' -> ['2078 Morris Ave','2084 Morris Ave','2090 Morris Ave']
    '1051-1057-1061-1065 Boston Rd' -> same expansion
    """
    m = re.search(r"\b(\d{1,6}(?:-\d{1,6})+)\s+([A-Z].+)$", u)
    if not m:
        return [u]
    block = m.group(1)
    tail = m.group(2)
    parts = [p for p in re.split(r"-", block) if p]
    base = parts[0]
    expanded = []
    for p in parts:
        if len(p) < len(base):  # "2078-84" -> fill high-order digits from base
            p = base[:len(base)-len(p)] + p
        expanded.append(f"{p} {tail}")
    return expanded

def _split_multi(s: str) -> List[str]:
    s = SEP_PATTERN.sub(" | ", s)  # unify to pipe, then split
    tokens = [t.strip(" .") for t in s.split("|") if _norm_ws(t)]
    # Remove address-like fragments that are obviously units/notes only
    return [t for t in tokens if t and not re.match(r"^(APT|STE|FL|B#\d+)\b", t, flags=re.IGNORECASE)]

def build_candidates(row) -> List[Tuple[str, dict, str]]:
    """
    Returns list of (query, components, reason_tag)
    """
    a1 = _std_street(row.get("address_line1_clean"))
    a2 = _std_unit(row.get("address_line2_clean"))
    borough = _borough_from_fields(row.get("borough_clean"), row.get("county"), row.get("city_clean"))
    city = _usps_city_from_borough(row.get("city_clean"), borough)
    zip5 = _std_zip5(row.get("zip"))
    state = STATE_ABBR

    raw = _norm_ws(row.get("geocode_key") or row.get("address_1") or "")
    raw = _apply_common_fixes(raw.upper()) if raw else ""
    raw_parts = _split_multi(raw) if raw else []

    # Build an address seed from clean fields
    seeds = []
    if a1:
        seeds.append(a1)
    seeds.extend(_expand_number_ranges(x) for x in raw_parts)
    seeds = [y for x in seeds for y in (x if isinstance(x, list) else [x])]
    seeds = [s for s in seeds if s] or ([raw.title()] if raw else [])

    candidates = []
    for s in seeds:
        s_norm = _std_street(s)
        if not s_norm:
            continue
        if _is_pobox(s_norm):
            candidates.append((None, {}, "POBOX"))
            continue
        if not _has_street_name(s_norm):
            candidates.append((None, {}, "MISSING_STREET"))
            continue

        line = s_norm
        parts = [line]
        # Append unit if it looks like an apartment (rarely helpful for geocode but OK)
        # if a2: parts.append(a2)

        locality = city or (borough.title() if borough else None)
        tail = ", ".join([p for p in [locality, state, zip5] if p])
        query = f"{' '.join(parts)}, {tail}" if tail else " ".join(parts)

        comps = {"administrative_area": state}
        if locality: comps["locality"] = locality

        candidates.append((query, comps, "OK"))
    # Deduplicate, keep order
    seen = set()
    out = []
    for q, c, r in candidates:
        key = (q or r, tuple(sorted(c.items())))
        if key not in seen:
            seen.add(key)
            out.append((q, c, r))
    return out


In [ ]:
# Cleaned drafts
building_info["address_line1_clean"] = building_info["address_1"].apply(_std_street)
building_info["address_line2_clean"] = building_info["address_2"].apply(_std_unit)

building_info["borough_clean"] = [
    _borough_from_fields(b, c, ci)
    for b, c, ci in zip(building_info.get("borough"), building_info.get("county"), building_info.get("city"))
]
building_info["city_clean"] = [
    _usps_city_from_borough(ci, bn)
    for ci, bn in zip(building_info.get("city"), building_info["borough_clean"])
]
building_info["state_clean"] = STATE_ABBR
building_info["postal_code_5_clean"] = building_info.get("postal_code").apply(_std_zip5) if "postal_code" in building_info.columns else None

# Mark rows missing coords
building_info["needs_geocoding"] = building_info["latitude"].isna() | building_info["longitude"].isna()

# a stable address key to dedupe geocoding calls
def addr_key(r):
    parts = [
        _norm_ws(r["address_line1_clean"]) or "",
        _norm_ws(r["address_line2_clean"]) or "",
        _norm_ws(r["city_clean"]) or "",
        STATE_ABBR,
        _norm_ws(r["postal_code_5_clean"]) or "",
    ]
    return "|".join(parts).upper()

building_info["geocode_key"] = building_info.apply(addr_key, axis=1)

building_info.head(3)[[
    "property_id",'bbl',"address_1","address_2","city","borough","county",
    "address_line1_clean","address_line2_clean","city_clean","state_clean","postal_code_5_clean",
    "needs_geocoding","geocode_key"
]]


In [ ]:
print(building_info.shape)
building_info = building_info.dropna(subset=["address_1","address_2","city","postcode","county","borough",
                                             "latitude","longitude","address_line1_clean"],how='all')
print(building_info.shape)

In [ ]:
### Geocode The Null lat long 
with_coord = building_info[building_info["needs_geocoding"]==False]
print(with_coord.shape)
without_coord = building_info[building_info["needs_geocoding"]==True]
print(without_coord.shape)

In [ ]:
without_coord['city'][without_coord['city']=='Flushinig']='Flushing'
without_coord['borough_clean'][(without_coord['city'].isin(['Sunnyside', 'Astoria', 'Jackson Heights', 'Flushing', 'Rego Park',
                                                   'Forest Hills', 'Oakland Gardens', 'Bayside', 'Kew Gardens','flushing',
                                                   'Woodside', 'Flushinig', 'Larchmont', 'Ridgewood','Jamaica', 'Albertson'])
                                &(without_coord['borough_clean'].isnull()))]='QUEENS'
### Manual Fixes where possible 
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="3115 brighton 6th"))]="BROOKLYN"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="144-35/39 Sanford Avenue"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="183-11 Hillside Ave"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="103-30/26 68th Ave"))]="QUEENS"
without_coord["borough_clean"][((without_coord['borough_clean'].isnull())&
               (without_coord["address_1"]=="71-11 -71-23 162ND STREET"))]="QUEENS"
without_coord = without_coord[~without_coord['borough_clean'].isnull()]
print(without_coord.shape)

### Start of Geocoding 

In [ ]:
# ---- GEOCODING LOOP (with retries/candidates/NYC bias) ----
results, failed = [], []

def geocode_row(row):
    cands = build_candidates(row)
    # Always add *raw borough + state* fallback for named complexes (e.g., NYCHA) to try to at least anchor
    borough = _borough_from_fields(row.get("borough_clean"), row.get("county"), row.get("city_clean"))
    if borough:
        cands.append((f"{borough.title()}, NY", {"administrative_area": "NY", "locality": borough.title()}, "BOROUGH_ONLY"))

    for query, comps, tag in cands:
        if not query:
            continue
        try:
            resp = gmaps.geocode(
                query,
                region="us",
                components=comps,
                bounds=NYC_BOUNDS
            )
        except Exception as e:
            print("Geocode error:", e)
            resp = []

        if resp:
            # prefer rooftop or range_interpolated results
            best = sorted(resp, key=lambda r: {"ROOFTOP":0, "RANGE_INTERPOLATED":1}.get(r.get("geometry",{}).get("location_type",""), 2))[0]
            loc = best["geometry"]["location"]
            return loc["lat"], loc["lng"], tag, best.get("formatted_address"), best.get("place_id")

    return None, None, "NO_HIT", None, None

# assumes without_coord is a DataFrame with the relevant columns
for i, r in without_coord[["property_id",'bbl',"geocode_key","address_line1_clean","address_line2_clean","borough_clean","city_clean","county"]].drop_duplicates().iterrows():
    # keep your i >= guard if needed
    temp_df = pd.DataFrame([r])
    lat, lng, tag, faddr, pid = geocode_row(r)

    if lat is not None and lng is not None:
        temp_df["latitude"]  = lat
        temp_df["longitude"] = lng
        temp_df["geocode_tag"] = tag
        temp_df["formatted_address"] = faddr
        temp_df["place_id"] = pid
        results.append(temp_df)
    else:
        temp_df["geocode_tag"] = tag  # e.g., POBOX, MISSING_STREET, NO_HIT
        failed.append(temp_df)


In [ ]:
print(len(results))
print(len(failed)) ## Zero failures

In [ ]:
results_df = pd.concat(results)

In [ ]:
### Property ID Present Without_coord (Checking)
print(without_coord.shape)
without_coord_geo = without_coord.dropna(how='all',axis=1).merge(results_df, 
                                                                 on=["property_id",'bbl',"geocode_key","address_line1_clean","address_line2_clean","borough_clean","city_clean","county"],
                                                                 how='left')

without_coord_geo[without_coord_geo['latitude'].isnull()]


In [ ]:
building_info_geocoded= pd.concat([with_coord,without_coord_geo])

In [ ]:
### Limit the resutlsd based on Building-specific Lat Long, not borough centroids.
building_info_geocoded = building_info_geocoded[building_info_geocoded['geocode_tag'].isnull()]
### NEEED TO ADD THE BBL BY ADDRESS.(Where needed)
print(building_info_geocoded.shape)

In [ ]:
need_bbl = building_info_geocoded[((building_info_geocoded['bbl'].isnull())
                                   |(building_info_geocoded['bbl'].astype(str).str.contains(";")))]
print(need_bbl.shape)
has_bbl = building_info_geocoded[((~building_info_geocoded['bbl'].isnull())
                                  &(~building_info_geocoded['bbl'].astype(str).str.contains(";")))]
print(has_bbl.shape)

### Enriching the Geocoded Building Information with BBLs using another API 

In [ ]:
df = need_bbl.copy()
rows = []

with requests.Session() as s:
    for i, r in df.iterrows():
        addr = str(r.get("address_line1_clean", "")).strip()
        boro = str(r.get("borough_clean", "")).strip()
        if not addr:
            rows.append((i, None, None, None, None, None)); continue

        q = f"{addr}, {boro}, NY" if boro else f"{addr}, NY"
        try:
            resp = s.get("https://geosearch.planninglabs.nyc/v2/search",
                         params={"text": q, "size": 5}, timeout=20)
            feats = resp.json().get("features", [])
            # pick the best feature: prefer layer=='address' and same borough; else highest confidence
            best, best_score = None, (-1, -1.0)
            for f in feats:
                p = f.get("properties", {})
                layer = p.get("layer")
                conf  = float(p.get("confidence", 0) or 0)
                fb    = (p.get("borough") or ((p.get("addendum") or {}).get("pad") or {}).get("boroughName") or "")
                score = (2 if layer == "address" else 0) + (1 if boro and boro.upper() in str(fb).upper() else 0)
                if (score, conf) > best_score:
                    best, best_score = f, (score, conf)

            if best:
                props = best.get("properties", {})
                add   = (props.get("addendum") or {}).get("pad") or {}
                bbl   = add.get("bbl") or props.get("bbl")
                bin_  = add.get("bin") or props.get("bin")

                # normalize BBL to a 10-digit string
                if isinstance(bbl, str):
                    digits = re.sub(r"\D", "", bbl)
                    bbl = digits.zfill(10) if digits else None
                elif bbl is not None:
                    try: bbl = str(int(bbl)).zfill(10)
                    except: bbl = None

                rows.append((i, bbl, bin_, props.get("confidence"), props.get("label") or props.get("name"), q))
            else:
                rows.append((i, None, None, None, None, q))

        except Exception as e:
            rows.append((i, None, None, None, f"ERR:{type(e).__name__}", q))

        time.sleep(0.1)  # tiny throttle to be polite

# assemble + join back
out = pd.DataFrame(rows, columns=["__idx","BBL_from_api","BIN_from_api","confidence","match_label","query"]).set_index("__idx")

In [ ]:
## Putting all tegehter and cleaning 
need_bbl_enr = need_bbl.join(out)
building_info_geocoded= pd.concat([need_bbl_enr, has_bbl])

In [ ]:
## Cleaning up some
building_info_geocoded['bbl'] = building_info_geocoded['BBL_from_api'].combine_first(building_info_geocoded['bbl'])
building_info_geocoded = building_info_geocoded.drop(columns=["BBL_from_api","BIN_from_api","confidence"])
building_info_geocoded = building_info_geocoded.dropna(how='all',axis=1)

### Pulling in Pluto Data for Num Floors and Height etc`

In [ ]:
# SOURCE # https://www.nyc.gov/content/planning/pages/resources/datasets/mappluto-pluto-change#mappluto
# ===================== CONFIG =====================
SRC   = r"C:/Users/johnf/Downloads/nyc_mappluto_25v3_fgdb/MapPLUTO25v3.gdb"
LAYER = "MapPLUTO_25v3_clipped"

# PLUTO columns you need (no geometry necessary)
PLUTO_COLS = ["BBL","LandUse","NumFloors","UnitsRes","BldgClass","Block","Borough","BoroCode"]

# How many distinct BLOCKS per chunk (per borough)
BLOCKS_PER_CHUNK = 2000  # tune: 1000–4000 are typical sweet spots

# ===================== HELPERS =====================
def norm_bbl_series(s: pd.Series) -> pd.Series:
    """Normalize to 10-digit BBL strings; handle '1234567890.0' correctly."""
    # Convert to string & trim
    s = s.astype(str).str.strip()

    # If it looks like 'digits.0' or 'digits.00', drop the decimal part
    s = s.str.replace(r"\.0+$", "", regex=True)

    # Remove any remaining non-digits, then pad to 10
    s = s.str.replace(r"\D", "", regex=True).str.zfill(10)

    # Turn "0000000000" into NA
    return s.mask(s.eq("0000000000"))

def boro_code_from_bbl_str(bbl: str) -> int:
    """First digit of BBL is borough code 1..5."""
    return int(bbl[0]) if isinstance(bbl, str) and bbl and bbl[0].isdigit() else None

def block_from_bbl_str(bbl: str) -> int:
    """Digits 2..6 (5 digits) are block."""
    try:
        return int(bbl[1:6])
    except Exception:
        return None

def pull_where(where: str, cols: list[str]) -> pd.DataFrame:
    """Run filtered attribute read; returns pandas.DataFrame (no geometry)."""
    return pyogrio.read_dataframe(
        SRC, layer=LAYER,
        columns=[c for c in cols if c != "geometry"],
        read_geometry=False,
        where=where,
    )

In [ ]:
# ===================== 1) LL84 BBL PREP =====================
ll84 = building_info_geocoded.copy()
ll84["bbl"] = norm_bbl_series(ll84["bbl"])
ll84 = ll84[ll84["bbl"].notna()].copy()

bbls = ll84["bbl"].unique().tolist()
print(f"[prep] Unique LL84 BBLs: {len(bbls):,}")

# ===================== 2) PULL PLUTO BY BBL IN(...) BATCHES =====================
parts = []
t0 = time.time()

BATCH = 800   # tune if needed
total_chunks = math.ceil(len(bbls) / BATCH)
print(f"[plan] {total_chunks} PLUTO chunks via BBL IN(...) batches")

for ci in range(total_chunks):
    chunk = bbls[ci*BATCH:(ci+1)*BATCH]

    # We already know "BBL = '...'" works, so quote as text
    in_list = ",".join(f"'{bb}'" for bb in chunk)
    where = f"BBL IN ({in_list})"

    part = pull_where(where, PLUTO_COLS)
    print(
        f"[pull] chunk {ci+1}/{total_chunks} | requested {len(chunk):,} BBLs "
        f"| pulled {len(part):,}"
    )

    if not part.empty:
        part["BBL"] = norm_bbl_series(part["BBL"])
        parts.append(part)

elapsed = time.time() - t0
pluto_by_bbl = (
    pd.concat(parts, ignore_index=True) if parts
    else pd.DataFrame(columns=PLUTO_COLS)
)

print(f"[done] Kept PLUTO rows: {len(pluto_by_bbl):,} in {elapsed:,.1f}s")

# ===================== 3) MERGE BACK TO LL84 + FILTER UNDER 5 STORIES =====================
ll84_enriched = ll84.merge(
    pluto_by_bbl.drop_duplicates("BBL"),
    left_on="bbl",
    right_on="BBL",
    how="left"
)

ll84_enriched["stories"] = ll84_enriched["NumFloors"]

res_codes = {"01", "02", "03", "04"}
is_res = (
    ll84_enriched["LandUse"].astype(str).isin(res_codes)
    | (ll84_enriched["UnitsRes"].fillna(0) > 0)
    | ll84_enriched.get(
        "BldgClass",
        pd.Series(index=ll84_enriched.index, dtype="object")
      ).fillna("").str.startswith(("A", "B"))
)

under5 = ll84_enriched[
    is_res
    & ll84_enriched["stories"].notna()
    & (ll84_enriched["stories"] < 5)
].copy()

print(
    f"[summary] LL84 rows: {len(ll84_enriched):,} | "
    f"Residential <5 stories: {len(under5):,}"
)


### Spatial joining with CT to pull ct into the buildings data. 

In [ ]:
CT2010_URL = "https://data.cityofnewyork.us/resource/bmjq-373p.geojson?$limit=50000"
resp = requests.get(CT2010_URL, timeout=60)
resp.raise_for_status()

ct2010 = gpd.read_file(io.BytesIO(resp.content)).to_crs(2263)
ct2010.head()

### Adding CT to buildigns

In [ ]:
# --- 1) Points from your geocoded table ---
# assumes working_residential_geo has columns: property_id, latitude, longitude
pts = gpd.GeoDataFrame(
    ll84_enriched[["property_id", "latitude", "longitude"]].copy(),
    geometry=gpd.points_from_xy(
        ll84_enriched["longitude"], ll84_enriched["latitude"]
    ),
    crs=4326  # your geocodes are WGS84
)

# project points to match the CT layer (your CT layer is already to_crs(2263))
pts_2263 = pts.to_crs(2263)

# --- 2) Keep only what you need from the CT layer ---
# 'ct2010' and/or 'boroct2010' are the usual tract IDs in that NYC layer
ct_keep = ct2010[["ct2010", "boroct2010", "boroname", "geometry"]].copy()

# --- 3) Spatial join: which tract polygon contains each point ---
joined = gpd.sjoin(
    pts_2263,
    ct_keep,
    how="left",
    predicate="within"   # points that fall inside a tract polygon
)

# If a property_id appears multiple times (duplicates), keep the first tract hit
joined = joined.sort_index().drop_duplicates(subset=["property_id"])

# --- 4) Bring tract columns back to your original dataframe ---
cols_to_add = ["ct2010", "boroct2010", "boroname"]
ll84_enriched_ct = ll84_enriched.merge(
    joined[["property_id"] + cols_to_add],
    on="property_id",
    how="left"
)

#### Pulling in the Building Footprionts of the buildings in the LL84 Data 

In [ ]:
limit = 50000
offset = 0
chunks = []

while True:
    url = f"https://data.cityofnewyork.us/resource/5zhs-2jue.geojson?$limit={limit}&$offset={offset}&$order=:id"
    r = requests.get(url, timeout=120); r.raise_for_status()
    g = gpd.read_file(io.BytesIO(r.content))
    if g.empty:
        break
    if g.crs is None:
        g = g.set_crs(4326)
    chunks.append(g)
    offset += limit
    # optional progress
    print(f"Fetched {len(g)} rows (total {sum(len(c) for c in chunks)})")
    if len(g) < limit:
        break
    time.sleep(0.2)  

building_ft_prnt = gpd.GeoDataFrame(pd.concat(chunks, ignore_index=True), crs=4326)

# De-dup by Socrata row id if present
if ':id' in building_ft_prnt.columns:
    building_ft_prnt = building_ft_prnt.drop_duplicates(subset=':id')

# Project to StatePlane feet for spatial ops
building_ft_prnt_2263 = building_ft_prnt.to_crs(2263)

print(building_ft_prnt_2263.shape, building_ft_prnt_2263.crs)
building_ft_prnt_2263.head()


In [ ]:
### Very laerge df so limiting to those BBLs that exist in my data for latlong / LL84
ll84_enriched_ct['bbl'] = ll84_enriched_ct['bbl'].astype(int).astype(str)
print(building_ft_prnt_2263.shape)
building_footprint_lim = building_ft_prnt_2263[building_ft_prnt_2263["base_bbl"].isin(ll84_enriched_ct["bbl"].unique())]
print(building_footprint_lim.shape)

In [ ]:
bbl_counts_ftpnt = building_footprint_lim.groupby(['base_bbl']).agg({"last_edited_date":'nunique'}).reset_index()
bbl_counts_ftpnt.reset_index(drop=True,inplace=True)

single = bbl_counts_ftpnt[bbl_counts_ftpnt['last_edited_date']==1]
print(single.shape)
mult = bbl_counts_ftpnt[bbl_counts_ftpnt['last_edited_date']>1]
print(mult.shape)

### Limtiing the df 
building_ft_prnt_2263_mult = building_footprint_lim[building_footprint_lim['base_bbl'].isin(mult['base_bbl'].unique())]
building_ft_prnt_2263_single= building_footprint_lim[building_footprint_lim['base_bbl'].isin(single['base_bbl'].unique())]

## Extracing the year for mult
building_ft_prnt_2263_mult['year'] = building_ft_prnt_2263_mult["last_edited_date"].dt.year

In [ ]:
mult_year = []
single_year = []
for bbl in building_ft_prnt_2263_mult['base_bbl'].unique():
    temp_df = building_ft_prnt_2263_mult[building_ft_prnt_2263_mult['base_bbl']==bbl]
    min_year = temp_df['year'].min()
    print(min_year)
    max_year = temp_df['year'].max()
    print(max_year)
    if min_year==max_year:
        single_year.append(temp_df[temp_df["last_edited_date"]==temp_df["last_edited_date"].max()])
    else: 
        if min_year>=2017:
            single_year.append(temp_df[temp_df["last_edited_date"]==temp_df["last_edited_date"].min()])
        else: 
            if max_year ==2017:
                sub_df = temp_df[temp_df['year']!=2017]
                if sub_df['year'].max()<=2010:
                    selection = sub_df[sub_df['last_edited_date'] == sub_df['last_edited_date'].max()]
                    if selection.shape[0]>1:
                        ## Taking the largest polygon 
                        single_year.append(selection[selection['shape_area'] == selection['shape_area'].max()])
                    else:
                        single_year.append(selection)
                else:
                    print("max year 2017, and there are other interim years. ")
                    for yy in range(2010,2018):
                        print(yy)
                        year_lim_temp= temp_df[temp_df['year']<=yy]
                        year_lim_temp = year_lim_temp[year_lim_temp['last_edited_date']==year_lim_temp['last_edited_date'].max()]
                        print(year_lim_temp.shape)
                        if year_lim_temp.shape[0]>1:
                            ## Again takign the largest
                            year_lim_temp = year_lim_temp[year_lim_temp['shape_area'] == year_lim_temp['shape_area'].max()]
                        year_lim_temp['year_manual']=yy
                        mult_year.append(year_lim_temp)
            else:
                print("max_year>2017")
                for yy in range(2010,2018):
                    print(yy)
                    year_lim_temp= temp_df[temp_df['year']<=yy]
                    year_lim_temp = year_lim_temp[year_lim_temp['last_edited_date']==year_lim_temp['last_edited_date'].max()]
                    print(year_lim_temp.shape)
                    if year_lim_temp.shape[0]>1:
                        ## Again takign the largest
                        year_lim_temp = year_lim_temp[year_lim_temp['shape_area'] == year_lim_temp['shape_area'].max()]
                    year_lim_temp['year_manual']=yy
                    mult_year.append(year_lim_temp)

In [ ]:
single_year_df = pd.concat(single_year)
single_bbls_footprint = pd.concat([building_ft_prnt_2263_single,single_year_df.drop(columns=['year'])])
mult_year_df = pd.concat(mult_year)

In [ ]:
### Two step join via BBL 
ll84_footprint_1 = ll84_enriched_ct.merge(single_bbls_footprint, how='left', left_on=['bbl'], right_on=['base_bbl'])
success1 = ll84_footprint_1[~ll84_footprint_1['base_bbl'].isnull()]
print(success1.shape)
not_success = ll84_footprint_1[ll84_footprint_1['base_bbl'].isnull()].dropna(how='all',axis=1)
print(not_success.shape)
#### success1 df is finalized building-specific information with the building footprint joined 
#### not_sccess df is the muiltiple year bbls, with changes in footprint for specific years, needs to be joined to year-specific energy building data before lidar spatial join. 

### Have Building Information joined with and the Canopy information, need to join back with the energy usage data (LL84)for year over year data

In [ ]:
### Limiting Energy Data tp those that have "Whole Building" metered eneryg area, but extrapolating for earlier years. 
residential3_2010_2017_lim1 = residential3_2010_2017[(
    ((residential3_2010_2017["metered_areas_energy"]=="Whole Building")&(residential3_2010_2017["year_ending_year"]>=2012))
     |(((residential3_2010_2017["metered_areas_energy"]=="Whole Building")|(residential3_2010_2017["metered_areas_energy"].isnull()))
       &(residential3_2010_2017["year_ending_year"]<=2012))
    )]
### Ensuring no relevant alerts for Energy Related meters 
residential3_2010_2017_lim2 = residential3_2010_2017_lim1[(
    ((residential3_2010_2017_lim1['alert_energy_no_meters'].isnull())|(residential3_2010_2017_lim1['alert_energy_no_meters']=='Ok'))
    &((residential3_2010_2017_lim1['alert_energy_meter_has_less'].isnull())|(residential3_2010_2017_lim1['alert_energy_meter_has_less']=='Ok'))
    &((residential3_2010_2017_lim1['alert_energy_meter_has_gaps'].isnull())|(residential3_2010_2017_lim1['alert_energy_meter_has_gaps']=='Ok'))
    &((residential3_2010_2017_lim1['alert_energy_meter_has'].isnull())|(residential3_2010_2017_lim1['alert_energy_meter_has']=='Ok'))
    &((residential3_2010_2017_lim1['alert_energy_meter_has_single'].isnull())|(residential3_2010_2017_lim1['alert_energy_meter_has_single']=='Ok'))
    &((residential3_2010_2017_lim1['alert_meter_has_less_than'].isnull())|(residential3_2010_2017_lim1['alert_meter_has_less_than']=='Ok'))
    &((residential3_2010_2017_lim1['alert_no_meters_are_associated'].isnull())|(residential3_2010_2017_lim1['alert_no_meters_are_associated']=='Ok'))
    &((residential3_2010_2017_lim1['alert_meter_has_overlaps'].isnull())|(residential3_2010_2017_lim1['alert_meter_has_overlaps']=='Ok'))
)]
### Dropping filtered columns 
residential3_2010_2017_lim2 = residential3_2010_2017_lim2.drop(columns=["metered_areas_energy","alert_energy_no_meters","alert_energy_meter_has_less",
                                                                        "alert_energy_meter_has_gaps","alert_energy_meter_has","alert_energy_meter_has_single",
                                                                        "alert_meter_has_less_than","alert_no_meters_are_associated","alert_meter_has_overlaps"])

### limiting to where number_of_buildings is 1 to keep simple, some years onlu have null values. 
residential3_2010_2017_lim3 = residential3_2010_2017_lim2[(
    ((residential3_2010_2017_lim2["number_of_buildings"]==1)&(residential3_2010_2017_lim2['year_ending_year']>=2013))
    |(((residential3_2010_2017_lim2["number_of_buildings"]==1)|(residential3_2010_2017_lim2["number_of_buildings"].isnull()))
      &(residential3_2010_2017_lim2['year_ending_year']<2013))
)]
### Generalized Way to limit to mutifamily residential buildings
multifamily_residential_cols = [i for i in residential3_2010_2017_lim3.columns if 'multifamily_' in i]
residential3_2010_2017_lim4 = residential3_2010_2017_lim3.dropna(subset=multifamily_residential_cols, how="all")

### Further Facility Type / Property Use Limiting
residential3_2010_2017_lim4 = residential3_2010_2017_lim4[(
    ((residential3_2010_2017_lim4["year_ending_year"]<2012) & (residential3_2010_2017_lim4["facility_type"]=='Multifamily Housing'))
    |((residential3_2010_2017_lim4["year_ending_year"]>=2012)&(residential3_2010_2017_lim4["facility_type"].isnull()))
)]

### Only Non estimated data, but also many nulls in this field
residential3_2010_2017_lim4 = residential3_2010_2017_lim4[(
    ((residential3_2010_2017_lim4['estimated_data_flag'].astype(str).str.upper().str.strip()=='NO')
     |(residential3_2010_2017_lim4['estimated_data_flag'].isnull()))
    &((residential3_2010_2017_lim4["temporary_data_flag"].isnull())
      |(residential3_2010_2017_lim4["temporary_data_flag"].astype(str).str.upper().str.strip()=='NO'))
    &((residential3_2010_2017_lim4["estimated_data_flag_1"].isnull())
      |(residential3_2010_2017_lim4["estimated_data_flag_1"].astype(str).str.upper().str.strip()=='NO'))
    &((residential3_2010_2017_lim4["estimated_values_energy"].isnull())
      |(residential3_2010_2017_lim4["estimated_values_energy"].astype(str).str.upper().str.strip()=='NO'))
    &((residential3_2010_2017_lim4["alert_property_has_no_uses"].isnull())
      |(residential3_2010_2017_lim4["alert_property_has_no_uses"].astype(str).str.upper().str.strip()=='OK'))
    &((residential3_2010_2017_lim4["default_values"].isnull())
      |(residential3_2010_2017_lim4["default_values"].astype(str).str.upper().str.strip()=='NO'))
    &((residential3_2010_2017_lim4["temporary_values"].isnull())
      |(residential3_2010_2017_lim4["temporary_values"].astype(str).str.upper().str.strip()=='NO'))
    
)]

## Dropping more columns already filtered on, but not needed for this analysis, also other columns not needed 9Less Noise to sift through for consolidation)
water_cols = [i for i in residential3_2010_2017_lim4.columns if 'water' in i]
national_metric_cols = [i for i in residential3_2010_2017_lim4.columns if 'national' in i]
flag_cols = [i for i in residential3_2010_2017_lim4.columns if 'flag' in i]
drinking_water = [i for i in residential3_2010_2017_lim4.columns if 'potable' in i]
electricity = [i for i in residential3_2010_2017_lim4.columns if 'electricity' in i] # looking at total energy for buildings not elec. (varies percentage by building)
other_specific_types = [i for i in residential3_2010_2017_lim4.columns if ('fuel' in i or 'steam' in i or 'diesel' in i or 'natural' in i )]
financial = [i for i in residential3_2010_2017_lim4.columns if ('_cost' in i or 'cost_' in i or 'savings' in i )]
greentech = [i for i in residential3_2010_2017_lim4.columns if 'green' in i]
emissions = [i for i in residential3_2010_2017_lim4.columns if 'emission' in i]


residential3_2010_2017_lim4 = residential3_2010_2017_lim4.drop(columns=list(set(["number_of_buildings",'estimated_values_energy']
                                                                                +multifamily_residential_cols
                                                                                +water_cols
                                                                                +national_metric_cols
                                                                                +flag_cols
                                                                                + drinking_water
                                                                                + electricity
                                                                                +other_specific_types
                                                                                +financial
                                                                                +greentech
                                                                                +emissions
                                                                               )))
## Droping all that are just null values for all entries
residential3_2010_2017_lim4 = residential3_2010_2017_lim4.dropna(how='all',axis=1)


In [ ]:
## Taking a look 
print(residential3_2010_2017_lim4.shape)
residential3_2010_2017_lim4.groupby(["year_ending_year"]).agg({"property_id":"nunique",'bbl':"nunique"})

In [ ]:
### Limiting the residential3_2010_2017_lim4 df to those instances that buildinga are present for ALL 7 years
property_id_2010 = residential3_2010_2017_lim4["property_id"][((residential3_2010_2017_lim4['year_ending_year']==2010)
                                                               &(~residential3_2010_2017_lim4["property_id"].isnull()))].unique()
bbl_2010 = residential3_2010_2017_lim4["bbl"][((residential3_2010_2017_lim4['year_ending_year']==2010)
                                                               &(~residential3_2010_2017_lim4["bbl"].isnull()))].unique()
consistant_presence = []
for year in residential3_2010_2017_lim4["year_ending_year"].unique():
    print(year)
    temp_lim_year = residential3_2010_2017_lim4[residential3_2010_2017_lim4["year_ending_year"]==year].dropna(how='all',axis=1)
    print(temp_lim_year.shape)
    if year ==2010:
        consistant_presence.append(temp_lim_year)
        continue
    else:
        temp_lim = temp_lim_year[((temp_lim_year["bbl"].isin(bbl_2010))|(temp_lim_year["property_id"].isin(property_id_2010)))]
        print(temp_lim.shape)
        print(f" Dropped {temp_lim_year.shape[0]-temp_lim.shape[0]} rows")
        consistant_presence.append(temp_lim)
all_building_energy_2010_2017 = pd.concat(consistant_presence)

In [ ]:
## Finalizing the Column names 
all_building_energy_2010_2017[all_building_energy_2010_2017['year_ending_year']==2010].dropna(how='all',axis=1).columns

In [ ]:
rename_2010 = {
    # Building area → canonical GFA
    "total_floor_space_sq_ft": "gfa_building_ft2",

    # Energy: EUI & totals
    "current_site_energy_intensity": "site_eui_kbtu_ft",
    "current_total_site_energy": "site_energy_use_kbtu",
    "target_site_energy_intensity": "target_site_eui_kbtu_ft",
    "current_total_source_energy": "source_energy_use_kbtu",

    # Normalized site energy index (not kBtu, but useful as separate indicator)
    "current_weather_normalized": "weather_normalized_site_energy_index",

    # Water & on-site energy
    "total_indoor_and_outdoor": "total_water_use_kgal",
    "current_total_on_site": "on_site_energy_use_kbtu",
    # keep current_source_energy as-is or drop it; you already have source_energy_use_kbtu above
}
rename_2011 = {
    # Building area
    "total_floor_space_sq_ft": "gfa_building_ft2",

    # Last modified → canonical
    "last_modified_date": "last_modified_date_property",

    # Energy: EUI & totals
    "current_site_energy_intensity": "site_eui_kbtu_ft",
    "current_total_site_energy": "site_energy_use_kbtu",
    "target_site_energy_intensity": "target_site_eui_kbtu_ft",
    "current_total_source_energy": "source_energy_use_kbtu",

    # Normalized site energy index
    "current_weather_normalized": "weather_normalized_site_energy_index",

    # Optional: you can keep current_source_energy as-is or ignore it
}
rename_2012 = {
    # State field
    "state_province": "state",

    # Building area
    "property_floor_area_buildings": "gfa_building_ft2",

    # Last modified → canonical
    "date_property_last_modified": "last_modified_date_property",

    # Energy fields already use canonical names:
    #   site_eui_kbtu_ft
    #   weather_normalized_site_eui
    #   site_energy_use_kbtu
    #   source_eui_kbtu_ft
    #   weather_normalized_source
    #   source_energy_use_kbtu
    #   target_site_eui_kbtu_ft
}
rename_2013 = {
    # Building area
    "property_floor_area_building": "gfa_building_ft2",

    # Last modified
    "date_property_last_modified": "last_modified_date_property",

    # Everything else you listed (energy) is already in canonical form:
    #   site_eui_kbtu_ft
    #   weather_normalized_site_eui
    #   site_energy_use_kbtu
    #   weather_normalized_site_energy
    #   source_eui_kbtu_ft
    #   weather_normalized_source
    #   source_energy_use_kbtu
    # plus the various weather_normalized_site* convenience columns
}
rename_2014 = {
    # Building area
    "property_floor_area_building": "gfa_building_ft2",

    # Energy fields already canonical names as in 2013
}
rename_2015 = {
    # Building area: self-reported & EPA-calculated
    "property_gfa_self_reported": "gfa_building_ft2",
    "property_gfa_epa_calculated": "gfa_epa_total_ft2",       # bldg + parking
    "property_gfa_epa_calculated_1": "gfa_epa_buildings_ft2", # buildings only

    # Last modified
    "date_property_last_modified": "last_modified_date_property",

    # (Energy columns already have canonical names)
}
rename_2016 = {
    # Building area: self-reported & calculated
    "property_gfa_self_reported": "gfa_building_ft2",
    "property_gfa_calculated": "gfa_epa_total_ft2",       # equivalent to EPA-calculated total
    "property_gfa_calculated_1": "gfa_epa_buildings_ft2", # buildings only

    # Last modified
    "date_property_last_modified": "last_modified_date_property",

    # Energy:
    #   site_eui_kbtu_ft
    #   weather_normalized_site_eui
    #   site_energy_use_kbtu
    #   weather_normalized_site_energy
    #   source_eui_kbtu_ft
    #   weather_normalized_source
    #   source_energy_use_kbtu
    #   weather_normalized_source_1 (duplicate variant)
    #   source_energy_use_adjusted
    #   source_eui_adjusted_to_current
    # all already match canon → no rename needed
}
rename_2017 = {
    # Building area
    "property_gfa_self_reported": "gfa_building_ft2",
    "property_gfa_calculated": "gfa_epa_total_ft2",
    "property_gfa_calculated_1": "gfa_epa_buildings_ft2",

    # Last modified already comes as last_modified_date_property for 2017 in your list.
    # No rename required.

    # Energy fields already match 2016’s canonical naming.
}
rename_maps = {
    2010: rename_2010,
    2011: rename_2011,
    2012: rename_2012,
    2013: rename_2013,
    2014: rename_2014,
    2015: rename_2015,
    2016: rename_2016,
    2017: rename_2017,
}


In [ ]:
# loop over years, rename columns, and collect
aligned_frames = []

for year in sorted(all_building_energy_2010_2017["year_ending_year"].unique()):
    print("Processing year:", year)
    temp_lim_year = all_building_energy_2010_2017[
        all_building_energy_2010_2017["year_ending_year"] == year
    ].copy()

    # apply the year-specific renames (if we have one)
    rename_dict = rename_maps.get(year, {})
    temp_lim_year = temp_lim_year.dropna(how='all',axis=1).rename(columns=rename_dict)
    if temp_lim_year.columns.duplicated().any():
        break
    print(temp_lim_year.columns)
    aligned_frames.append(temp_lim_year)

# 3) stitch them back together
all_building_energy_2010_2017_renamed = pd.concat(aligned_frames)

In [ ]:
final_cols = [
    # Core / IDs / location
    "year_ending_year",
    "property_id",
    "bbl",
    "address_1",
    "address_2",
    "city",
    "borough",
    "postcode",
    "state",

    # Data / timing
    "release_date",
    "generation_date",
    "current_energy_period_ending",
    "energy_current_date",
    "period_ending_date",
    "last_modified_date_property",

    # Building
    "year_built",
    "gfa_building_ft2",
    "gfa_epa_total_ft2",
    "gfa_epa_buildings_ft2",
    "largest_property_use_type_1",
    "costar_property_id",

    # Energy – core
    "site_eui_kbtu_ft",
    "weather_normalized_site_eui",
    "site_energy_use_kbtu",
    "weather_normalized_site_energy",
    "source_eui_kbtu_ft",
    "weather_normalized_source",
    "source_energy_use_kbtu",
    "target_site_eui_kbtu_ft",

    # Energy – early-year extras
    "weather_normalized_site_energy_index",
    "current_source_energy",
    "on_site_energy_use_kbtu",
    "total_water_use_kgal",

    # Extra normalized variants
    "weather_normalized_source_1",
    "weather_normalized_site",
    "weather_normalized_site_1",
    "weather_normalized_site_2",
    "weather_normalized_site_3",

    # Adjusted source metrics
    "source_energy_use_adjusted",
    "source_eui_adjusted_to_current",
]

all_building_energy_2010_2017_renamed = all_building_energy_2010_2017_renamed[final_cols]
# all_building_energy_2010_2017_renamed.to_csv("all_building_energy_2010_2017_renamed.csv",index=False)

In [ ]:
## Finally Joining back in the buidlings information

In [ ]:
all_building_energy_2010_2017_renamed

### Joining the energy building data and the Building Footprint data 

In [ ]:
## Joing with the footprints already 
success1
print(success1.shape)
### Needs to add footprints, but for year-specific ones 
not_success
print(not_success.shape)

### This is the footprint data for those that aren int success/ single bb; resolution data: mult_year_df

In [ ]:
### Multi Step join 
join_columns_energy = [i for i in all_building_energy_2010_2017_renamed.columns if i not in ['borough','postcode','state']]
all_building_energy_2010_2017_renamed

## PART 1 
final_building_energy_data1 = all_building_energy_2010_2017_renamed[join_columns_energy].merge(
    success1,
        how="inner",
        on=["property_id","bbl","address_1","address_2","city"])
print("final_building_energy_data1")
print(final_building_energy_data1.shape)
print("final_building_energy_data1 bbls")
print(len(final_building_energy_data1['bbl'].unique()))

print("not_success")
print(not_success.shape)
print("not_success bbls")
print(len(not_success['bbl'].unique()))

## Part 2 
final_building_energy_data2 = all_building_energy_2010_2017_renamed[join_columns_energy].merge(
    not_success,
        how="inner",
        on=["property_id","bbl","address_1","address_2","city"])
print("final_building_energy_data2")
print(final_building_energy_data2.shape)
print("final_building_energy_data2 bbls")
print(len(final_building_energy_data2['bbl'].unique()))

### Checking Mutli Stats 
print("mult_year_df")
print(mult_year_df.shape)
print('mult_year_df bbls')
print(len(mult_year_df['base_bbl'].unique()))

# final_building_energy_data1
final_building_energy_data3 = final_building_energy_data2.merge(
    mult_year_df,
    how='inner', 
    right_on = ['base_bbl','year_manual'], 
    left_on = ['bbl','year_ending_year'])
print(final_building_energy_data3.shape)
print(len(final_building_energy_data3['bbl'].unique()))


# final_building_energy_data1
final_building_energy_data4 = final_building_energy_data2.merge(
    mult_year_df,
    how='left', 
    right_on = ['base_bbl','year_manual'], 
    left_on = ['bbl','year_ending_year'])
print(final_building_energy_data4.shape)
print(len(final_building_energy_data4['bbl'].unique()))


In [ ]:
### putting it all together with waht was successful for multi year matching and singluelar values 
final_building_data = pd.concat([final_building_energy_data1,final_building_energy_data3])
final_building_data = final_building_data.drop(columns = ['year_manual','year'])

### SEMI FINAL WORKING DATASET WITH BUIDLGINDS
print(final_building_data.groupby(['year_ending_year']).agg({"bbl":"nunique"}))

In [ ]:
#### THREE WORKING DATA SETS FOR THE ANALYSIS 

### FIRST IS 2010 and 2017 years specifically to have more buildings for canopy analysis
master_2010_2017_bbl_list = [b for b in 
                             list(final_building_data['bbl'][final_building_data['year_ending_year'] ==2010].unique()) 
                             if b in list(final_building_data['bbl'][final_building_data['year_ending_year'] ==2017].unique())]
print(len(master_2010_2017_bbl_list))
building_final_2010_2017 = final_building_data[((final_building_data['year_ending_year'].isin([2010,2017]))
                                                &(final_building_data['bbl'].isin(master_2010_2017_bbl_list)))]
building_final_2010_2017.to_csv('building_final_2010_2017_20251116.csv',index=False)

### Second is only the BBLs that are present in every eyar, with a max possible number of 134 because od 2012 being so low.
years = range(2010, 2018)  # adjust as needed
bbl_sets = [set(final_building_data.loc[final_building_data['year_ending_year'] == y, 'bbl'].unique()) for y in years]
common_bbls = set.intersection(*bbl_sets)  # BBLs present in ALL years
common_bbls = list(common_bbls)           # back to list if you want
print(len(common_bbls)) ## 25 bbls 
building_final_all_yrs = final_building_data[final_building_data['bbl'].isin(common_bbls)]
print(building_final_all_yrs.shape)
building_final_all_yrs.to_csv('building_final_all_yrs_20251116.csv',index=False)


### OPTIONAL1 (third would be all bbls and buildings with energy data (reguardless of fotoprint data)
max_buildings_no_footprint = pd.concat([final_building_energy_data1,final_building_energy_data4])
years = range(2010, 2018)  # adjust as needed
bbl_sets = [set(max_buildings_no_footprint.loc[max_buildings_no_footprint['year_ending_year'] == y, 'bbl'].unique()) for y in years]
common_bbls = set.intersection(*bbl_sets)  # BBLs present in ALL years
common_bbls = list(common_bbls)           # back to list if you want
print(len(common_bbls)) ## 33  bbls 
max_buildings_no_footprint_all_years = max_buildings_no_footprint[max_buildings_no_footprint['bbl'].isin(common_bbls)]
max_buildings_no_footprint_all_years = max_buildings_no_footprint_all_years.drop(columns = ['year_manual','year'])
max_buildings_no_footprint_all_years.to_csv('max_buildings_no_footprint_all_years_20251116.csv',index=False)


max_buildings_no_footprint = pd.concat([final_building_energy_data1,final_building_energy_data4])
years = [2010, 2017]  # adjust as needed
bbl_sets = [set(max_buildings_no_footprint.loc[max_buildings_no_footprint['year_ending_year'] == y, 'bbl'].unique()) for y in years]
common_bbls = set.intersection(*bbl_sets)  # BBLs present in ALL years
common_bbls = list(common_bbls)           # back to list if you want
print(len(common_bbls)) ## 1664 bbls 
max_buildings_no_footprint_2010_2017 = max_buildings_no_footprint[max_buildings_no_footprint['bbl'].isin(common_bbls)]
max_buildings_no_footprint_2010_2017 = max_buildings_no_footprint_2010_2017.drop(columns = ['year_manual','year'])
max_buildings_no_footprint_2010_2017.to_csv('max_buildings_no_footprint_2010_2017_20251116.csv',index=False)

## Now That the building Data is finalized, need to add the Lidar for Canopy Data 

### Tree Canopy Joining via spatial join with 50 foot buffer

In [ ]:
# Path to your geodatabase (.gdb)
# https://github.com/CityOfNewYork/nyc-geo-metadata/blob/main/Metadata/Metadata_TreeCanopyChange.md
gdb_path = r"C:\Users\johnf\Downloads\Tree_Canopy_Change (1)\Tree_Canopy_Change\NYC_TreeCanopyChange_2010_2017.gdb"

# List all layers in the geodatabase
layers = fiona.listlayers(gdb_path)
print("Layers available:", layers)

# Read a specific layer into a GeoDataFrame
gdf = gpd.read_file(gdb_path, layer=layers[0])  # Replace [0] with your desired layer
print(gdf.head())

# Optional: save as shapefile or GeoJSON
# gdf.to_file("output.shp") 
# gdf.to_file("output.geojson", driver="GeoJSON")

canopy_change = gdf.copy()
# canopy_change
print(canopy_change.crs)

### Geodataframe conversion for building_final_2010_2017


In [ ]:
# starting from your existing df, e.g. building_final_2010_2017
bdf = building_final_2010_2017.copy()
# geometry already contains MultiPolygon objects, so just wrap as GeoDataFrame
buildings_gdf = gpd.GeoDataFrame(bdf, geometry="geometry", crs="EPSG:2263")  # CRS in feet
# 50-ft buffer around each building
buildings_buffered = buildings_gdf.copy()
buildings_buffered["geometry"] = buildings_buffered.geometry.buffer(50)

In [ ]:
# ensure canopy_gdf is in same CRS
canopy_change = canopy_change.to_crs(buildings_buffered.crs)

# 2. Compute actual intersection geometries
#    This creates one row per building–canopy overlap
intersections = gpd.overlay(
    buildings_buffered[['bbl', 'geometry']],  # keep bbl + geometry
    canopy_change,                               # keep all canopy attrs
    how='intersection'
)

# 3. Compute overlap area for each pair (units: ft² since EPSG:2263)
intersections['overlap_area'] = intersections.geometry.area

# 4. For each building (bbl), keep the row with the largest overlap_area
idx_max = intersections.groupby('bbl')['overlap_area'].idxmax()
largest_canopy_per_bldg = intersections.loc[idx_max].copy()

largest_canopy_per_bldg = largest_canopy_per_bldg.rename(
    columns={"geometry": "canopy_geom","Class":"canopy_change_class"})

buildings_with_canopy = buildings_buffered.merge(
    largest_canopy_per_bldg[["bbl", "canopy_geom", "canopy_change_class", "overlap_area"]],
    on="bbl",
    how="left"
)

In [ ]:
#Taking a look
buildings_with_canopy.groupby(['canopy_change_class','year_ending_year']).agg({'bbl':"nunique"})

### Geodataframe conversion for building_final_all_yrs


In [ ]:
# starting from your existing df, e.g. building_final_all_yrs
bdf = building_final_all_yrs.copy()
# geometry already contains MultiPolygon objects, so just wrap as GeoDataFrame
buildings_gdf = gpd.GeoDataFrame(bdf, geometry="geometry", crs="EPSG:2263")  # CRS in feet
# 50-ft buffer around each building
buildings_buffered = buildings_gdf.copy()
buildings_buffered["geometry"] = buildings_buffered.geometry.buffer(50)

# ensure canopy_gdf is in same CRS
canopy_change = canopy_change.to_crs(buildings_buffered.crs)

# 2. Compute actual intersection geometries
#    This creates one row per building–canopy overlap
intersections = gpd.overlay(
    buildings_buffered[['bbl', 'geometry']],  # keep bbl + geometry
    canopy_change,                               # keep all canopy attrs
    how='intersection'
)

# 3. Compute overlap area for each pair (units: ft² since EPSG:2263)
intersections['overlap_area'] = intersections.geometry.area

# 4. For each building (bbl), keep the row with the largest overlap_area
idx_max = intersections.groupby('bbl')['overlap_area'].idxmax()
largest_canopy_per_bldg = intersections.loc[idx_max].copy()

largest_canopy_per_bldg = largest_canopy_per_bldg.rename(
    columns={"geometry": "canopy_geom","Class":"canopy_change_class"})

buildings_with_largest_canopy = buildings_buffered.merge(
    largest_canopy_per_bldg,
    on='bbl',
    how='left',
    suffixes=('', '_canopy')
)

In [ ]:
#Taking a look
buildings_with_largest_canopy.groupby(['canopy_change_class','year_ending_year']).agg({'bbl':"nunique"})

In [ ]:
## Limited ll84_enriched_ct_canopy data for joining with the energy metrics
# ll84_enriched_ct_canopy.columns
ll84_enriched_ct_canopy[ll84_enriched_ct_canopy["borough"]=="STATEN IS"]="STATEN ISLAND"

## Striping those that are iffy, and keeping the solid ones. If needed i will clean later. 
print(ll84_enriched_ct_canopy.shape)
ll84_enriched_ct_canopy_limited = ll84_enriched_ct_canopy[(
    (ll84_enriched_ct_canopy["borough"]==ll84_enriched_ct_canopy["borough_clean"])
    &(## manhattan
        ((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "1") & (ll84_enriched_ct_canopy["borough"]=="MANHATTAN"))
    ##Bronx
        |((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "2") & (ll84_enriched_ct_canopy["borough"]=="BRONX"))
    #Brooklyn
         |((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "3") & (ll84_enriched_ct_canopy["borough"]=='BROOKLYN'))
    #Queens
        |((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "4") & (ll84_enriched_ct_canopy["borough"]=="QUEENS"))
    #Staten Island
    |((ll84_enriched_ct_canopy["bbl"].astype(str).str[0] == "5") & (ll84_enriched_ct_canopy["borough"]=="STATEN ISLAND"))
    # Null BBL
    |(ll84_enriched_ct_canopy["bbl"].isnull())
    ))]
print(ll84_enriched_ct_canopy_limited.shape)
### Dropped 1518 rows / buildings

In [ ]:
# ll84_enriched_ct_canopy.to_csv("ll84_enriched_ct_canopy.csv",index=False)

## Below is manual work for column alignment for multi year data.

In [ ]:
### 
column_dict = {}
for year in residential3_2010_2017_lim4['year_ending_year'].unique():
    print(year)
    temp_df = residential3_2010_2017_lim4[residential3_2010_2017_lim4['year_ending_year']==year].dropna(how='all',axis=1)
    print(list(temp_df.columns))
    column_dict[year]=list(temp_df.columns)

In [ ]:
### 2010
## Core for joining 
'year_ending_year',
'property_id',
'bbl',
'address_1',
'city',
"borough",
"postcode",
'address_2',
"state",
## Data Information
'release_date',
"last_modified_date",
"current_energy_period_ending",
"period_ending_date",
#Building
"total_floor_space_sq_ft",
## Energy Use
'current_energy_period_ending', 
'current_site_energy_intensity', 
'current_total_site_energy', 
'target_site_energy_intensity',
'current_source_energy', 
'current_total_source_energy', 
'current_weather_normalized', 
'total_indoor_and_outdoor', 
'current_total_on_site'
#---------------------------------------
## 2011
## Core for joining 
'year_ending_year',
'property_id',
'bbl',
'address_1',
'city',
"borough",
"postcode",
'address_2',
"state",
## Data Information
"release_date"
"current_energy_period_ending"

## building
"total_floor_space_sq_ft"
"year_built"
"last_modified_date"
# metering_configuration  ?

##energy
weather_normalized_site_eui
'current_energy_period_ending',
'current_site_energy_intensity',
'current_total_site_energy',
'target_site_energy_intensity',
'current_source_energy',
'current_total_source_energy',
'current_weather_normalized'
###-------------------------
##2012
'year_ending_year'
'property_id', 
'bbl'
 'address_1',
'city', 
 'borough',
 'address_2',
 'state_province'
## Data info
'generation_date'
'release_date', 
'energy_current_date'
#Building
'year_built',
'date_property_last_modified'
 'property_floor_area_buildings'

# Engergy
'site_eui_kbtu_ft',
'weather_normalized_site_eui',
'site_energy_use_kbtu',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',  
'target_site_eui_kbtu_ft',
### -------------------------
### 2013
## Core for joining
 'year_ending_year',
'property_id', 
 'bbl', 
'address_1', 
'city', 
 'borough',
'postcode',  
'address_2',
## data info 
'generation_date',
'release_date',

#building info
'year_built',  
 'date_property_last_modified'
'property_floor_area_building'
#energy
'site_eui_kbtu_ft',
'weather_normalized_site_eui',
'site_energy_use_kbtu',
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',
'weather_normalized_source_1', 
'weather_normalized_site',
'weather_normalized_site_1',
  'costar_property_id', 
###-------------------------------
## 2014
## Core for joining 
'year_ending_year',
'property_id',
'bbl',
  'address_1', 
'city',
'borough',
'postcode'
'address_2'
## Data Information
'generation_date',
'release_date'

## Buidling Info
'year_built',
'property_floor_area_building',
## Energy Use
 'site_eui_kbtu_ft', 
'weather_normalized_site_eui', 
'site_energy_use_kbtu', 
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source'
'source_energy_use_kbtu'
 'weather_normalized_source_1'
'weather_normalized_site',
'weather_normalized_site_1',
'weather_normalized_site_2',
'weather_normalized_site_3',
## 2015
'property_id',
'address_1', 
'city',
 'borough',
 'postcode'
 'address_2', 
## Core for joining 

## Data Information
'generation_date'
'release_date',
## Buidling Info
'largest_property_use_type_1',
'year_built',
'date_property_last_modified',
"property_gfa_self_reported"
 'property_gfa_epa_calculated', 
'property_gfa_epa_calculated_1'
## Energy Use
'site_eui_kbtu_ft',
'weather_normalized_site_eui',
'site_energy_use_kbtu',
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',
'weather_normalized_source_1', 
 'weather_normalized_site', 'weather_normalized_site_1', 'weather_normalized_site_2', 'weather_normalized_site_3',
#-----------------------------
## 2016
## Core for joining 
'property_id',
'bbl',
'address_1',
'city',
'borough'
 'postcode', 
'address_2',


## Data Information
'generation_date'
'release_date',
## Buidling Info
'largest_property_use_type_1',
'year_built',
 'property_gfa_self_reported', 
'property_gfa_calculated', 
'property_gfa_calculated_1',
'date_property_last_modified'
## Energy Use
'site_eui_kbtu_ft',
'weather_normalized_site_eui',
'site_energy_use_kbtu',
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',
'weather_normalized_source_1',
 'weather_normalized_site', 
'weather_normalized_site_1', 
'weather_normalized_site_2', 
'weather_normalized_site_3',
'source_energy_use_adjusted'
'source_eui_adjusted_to_current'
##### --------------------------------------
## 2017
## Core for joining
 'year_ending_year'
'property_id', 
'bbl'
'address_1',
'city',  
'borough',
'postcode', 
'address_2',  \

## Data Information
'generation_date'

## Buidling Info
 'largest_property_use_type_1', 
'year_built',
'property_gfa_self_reported',
'property_gfa_calculated', 
'property_gfa_calculated_1
'last_modified_date_property',
## Energy Use
'site_eui_kbtu_ft', 
'weather_normalized_site_eui', 
'site_energy_use_kbtu',
'weather_normalized_site_energy',
'source_eui_kbtu_ft',
'weather_normalized_source',
'source_energy_use_kbtu',
'weather_normalized_source_1', 
 'weather_normalized_site', 
'weather_normalized_site_1',
'weather_normalized_site_2',
'weather_normalized_site_3', 
 'source_energy_use_adjusted',
